# 抓取台股所有股票的6月營收

In [1]:
import pandas as pd
import requests
from io import StringIO
import time
def monthly_report(year, month):
    
    # 假如是西元，轉成民國
    if year > 1990:
        year -= 1911
    
    url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'_0.html'
    if year <= 98:
        url = 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(year)+'_'+str(month)+'.html'
    
    # 偽瀏覽器
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    
    # 下載該年月的網站，並用pandas轉換成 dataframe
    r = requests.get(url, headers=headers)
    r.encoding = 'big5'

    dfs = pd.read_html(StringIO(r.text), encoding='big-5')

    df = pd.concat([df for df in dfs if df.shape[1] <= 11 and df.shape[1] > 5])
    
    if 'levels' in dir(df.columns):
        df.columns = df.columns.get_level_values(1)
    else:
        df = df[list(range(0,10))]
        column_index = df.index[(df[0] == '公司代號')][0]
        df.columns = df.iloc[column_index]
    
    df['當月營收'] = pd.to_numeric(df['當月營收'], 'coerce')
    df = df[~df['當月營收'].isnull()]
    df = df[df['公司代號'] != '合計']
    
    # 偽停頓
    time.sleep(5)

    return df

In [2]:
year=111
month=6
res=monthly_report(year, month)

# 前100個的YoY min為51.3 

In [33]:
yoyrate=res.copy()
minYoY=yoyrate['去年同月增減(%)'].nlargest(100).min()
print('min YoY',minYoY)
yoyfilter=yoyrate['去年同月增減(%)']>minYoY
yoyrate=yoyrate[yoyfilter]
yoyrate.sort_values(['去年同月增減(%)'],ascending=False)

min YoY 51.3


,公司代號,公司名稱,備註,上月比較增減(%),上月營收,去年同月增減(%),去年當月營收,當月營收,前期比較增減(%),去年累計營收,當月累計營收
1,1438,三地開發,"111年累計銷售商品及土地455,714仟元,110年累計銷售商品及土地428仟元,故營收變化大。",11392275.00,4,7594816.66,6,455695,106375.23,428,455714
40,9906,欣巴巴,本月營收與上月之差異，係有個案已全數交屋過戶。累計營收較去年成長之差異，係有新個案銷售入帳所致。,-57.83,737097,2390423.07,13,310768,5262.80,40484,2171079
30,3056,總太,本月營收增加係因個案共好Melody完工交屋。,57513.73,3603,27253.05,7589,2075823,-41.49,3593209,2102110
38,5534,長虹,本月較去年同期增加，係因「雲端科技」、「交響苑」個案增加完成產權移轉予客戶認列收入所致。,171.26,266282,4033.98,17473,722332,418.83,983803,5104290
6,2722,夏都,110年6月因疫情三級警戒，營收銳減。,75.18,35357,1896.77,3102,61940,21.88,273147,332926
...,...,...,...,...,...,...,...,...,...,...,...
15,1447,力鵬,本月子公司因受原物料價格上漲及貿易量增加之影響，故營收增加。,-0.00,2998495,53.32,1955590,2998409,17.50,12007317,14109774
3,2345,智邦,客戶需求增加,12.31,6156162,53.06,4517191,6914045,26.32,27251362,34425741
29,6426,統新,業績成長,-10.74,64811,52.73,37873,57847,34.02,264532,354543
3,2331,精英,料況解決，提高出貨量所致,23.33,3485361,51.87,2830469,4298712,30.69,13944950,18225598


# 前100個的MoM的min為30.37

In [35]:
momrate=res.copy()
minMoM=momrate['上月比較增減(%)'].nlargest(100).min()
print('min MoM',minMoM)
momdi=momrate['上月比較增減(%)']>minMoM
momrate=momrate[momdi]
momrate=momrate.sort_values(['上月比較增減(%)'],ascending=False)
momrate['公司代號']

min MoM 30.37


1     1438
30    3056
20    4142
16    2527
41    9946
      ... 
2     2704
25    4555
6     2356
82    8249
0     1603
Name: 公司代號, Length: 99, dtype: object

#  分析股價與營收之關係(MoM)

In [37]:
import pandas_datareader as pdr
import datetime as datetime


#  取得前100筆MoM股票代號

In [66]:
t=momrate.set_index('公司代號')
a=[]
for i in range(99):
    a.append(t.index[i]+'.TW')
a[0]

'1438.TW'

In [69]:
start = datetime.datetime(2022,6,30) #from datatime
df = pdr.DataReader(a[0], 'yahoo', start=start)
df['Close']

Date
2022-06-30    31.400000
2022-07-01    30.700001
2022-07-04    31.400000
2022-07-05    32.000000
2022-07-06    32.000000
2022-07-07    32.000000
2022-07-08    32.049999
2022-07-11    33.099998
Name: Close, dtype: float64

In [71]:
df=[]
for i in range(99):
    df.append(pdr.DataReader(a[i], 'yahoo', start=start))

In [73]:
# label data
for i in range(99):
    df[i]['id']=a[i]

In [100]:
for i in range(99):
    
    if df[i]['rate'].mean>=0:
        with pd.ExcelWriter(f'good.xlsx', engine='xlsxwriter') as writer:
            df[i].to_excel(writer, sheet_name=df[i]['id'])
    else:
        with pd.ExcelWriter(f'bad.xlsx', engine='xlsxwriter') as writer:
            df[i].to_excel(writer, sheet_name=df[i]['id'])
    for j in range(7):
        try:
            value=((df[i]['Close'][j+1]-df[i]['Close'][j])/df[i]['Close'][j])*100
            if j==0:
                df[i]['rate'][j]=0
            df[i]['rate'][j+1]=value
        except IndexError:
#             print(df[i]['id'][j],df[i]['Close'][j])
            if j==0:
                df[i]['rate'][j]=0
        

D:\anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
D:\anaconda\envs\py36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


# 漲幅平均有上漲的有51隻 下降的有49隻

In [119]:
goodc=0
badc=0
with pd.ExcelWriter(f'goodMoM100.xlsx', engine='xlsxwriter') as writer:
    for i in range(99):
        if df[i]['rate'].mean()>=0:
                goodc=goodc+1
                df[i].to_excel(writer, sheet_name=df[i]['id'][0])
with pd.ExcelWriter(f'badMoM100.xlsx', engine='xlsxwriter') as writer2:
    for i in range(99):
        if df[i]['rate'].mean()<0:
                badc=badc+1
                df[i].to_excel(writer2, sheet_name=df[i]['id'][0])
counter=0
with pd.ExcelWriter(f'goodMoM100.xlsx', engine='xlsxwriter') as writer:
    for i in range(99):
        if df[i]['rate'].mean()>=0:
            df[i].to_excel(writer, sheet_name=df[i]['id'][0], index=True, na_rep='NaN')
            # Auto-adjust columns' width
            for column in df[i]:
                column_width = 20
                col_idx = df[i].columns.get_loc(column)
                writer.sheets[df[i]['id'][0]].set_column(col_idx, col_idx, column_width)
with pd.ExcelWriter(f'badMoM100.xlsx', engine='xlsxwriter') as writer2:
    for i in range(99):
        if df[i]['rate'].mean()<0:
            df[i].to_excel(writer2, sheet_name=df[i]['id'][0], index=True, na_rep='NaN')
            # Auto-adjust columns' width
            for column in df[i]:
                column_width = 20
                col_idx = df[i].columns.get_loc(column)
                writer2.sheets[df[i]['id'][0]].set_column(col_idx, col_idx, column_width)

In [121]:
# 'good',goodc,'bad',badc
for i in range(99):
    if df[i]['rate'].mean()>=3:
        print(df[i]['id'])

Date
2022-06-30    6796.TW
2022-07-01    6796.TW
2022-07-04    6796.TW
2022-07-05    6796.TW
2022-07-06    6796.TW
2022-07-07    6796.TW
2022-07-08    6796.TW
2022-07-11    6796.TW
Name: id, dtype: object
Date
2022-06-30    2419.TW
2022-07-01    2419.TW
2022-07-04    2419.TW
2022-07-05    2419.TW
2022-07-06    2419.TW
2022-07-07    2419.TW
2022-07-08    2419.TW
2022-07-11    2419.TW
Name: id, dtype: object


# 平均漲幅最高的為6796 2419

In [127]:
highcodi=momrate['公司代號']=='6796'
momrate[highcodi]

,公司代號,公司名稱,備註,上月比較增減(%),上月營收,去年同月增減(%),去年當月營收,當月營收,前期比較增減(%),去年累計營收,當月累計營收
25,6796,晉弘,-,60.68,28645,14.32,40260,46029,15.01,198910,228768


In [126]:
highcodi=momrate['公司代號']=='2419'
momrate[highcodi]

,公司代號,公司名稱,備註,上月比較增減(%),上月營收,去年同月增減(%),去年當月營收,當月營收,前期比較增減(%),去年累計營收,當月累計營收
4,2419,仲琦,配合客戶需求出貨量增加,54.86,887599,96.2,700587,1374598,6.29,5155444,5480133


# 單日漲幅較高的(>6%)有24隻

In [138]:
a=[]
for i in range(99):
    for j in range(8):
        try:
            if df[i]['rate'][j]>=6:
                print(df[i]['id'][j])
                a.append(i)
                break
        except IndexError:
            break
with pd.ExcelWriter(f'singlehighMoM100.xlsx', engine='xlsxwriter') as writer:
    for i in a:
        df[i].to_excel(writer, sheet_name=df[i]['id'][0])
with pd.ExcelWriter(f'singlehighMoM100.xlsx', engine='xlsxwriter') as writer:
    for i in a:
        df[i].to_excel(writer, sheet_name=df[i]['id'][0], index=True, na_rep='NaN')
            # Auto-adjust columns' width
        for column in df[i]:
            column_width = 20
            col_idx = df[i].columns.get_loc(column)
            writer.sheets[df[i]['id'][0]].set_column(col_idx, col_idx, column_width)

4142.TW
1529.TW
8478.TW
3229.TW
6226.TW
6515.TW
6289.TW
3062.TW
3380.TW
2498.TW
6796.TW
4770.TW
3535.TW
2419.TW
3046.TW
1708.TW
1533.TW
5222.TW
2457.TW
3406.TW
5258.TW
1435.TW
2493.TW
3094.TW


In [139]:
import openpyxl

# 取得excel data(多個sheet)

In [146]:
wb = openpyxl.load_workbook("goodMoM100.xlsx")
test=[]
file_name="goodMoM100.xlsx"
#give the full path of the file here
size=wb.sheetnames
for i in size:
    test.append(pd.read_excel(file_name, sheet_name=i,engine='openpyxl'))
test[1]

,Date,High,Low,Open,Close,Volume,Adj Close,id,rate
0,2022-06-30,41.950001,38.650002,41.750000,39.099998,8788220,39.099998,4142.TW,0.000000
1,2022-07-01,38.900002,36.299999,38.299999,36.400002,5327002,36.400002,4142.TW,-6.905363
2,2022-07-04,37.400002,35.650002,36.400002,36.000000,2349335,36.000000,4142.TW,-1.098905
3,2022-07-05,38.049999,36.400002,36.599998,38.049999,4146326,38.049999,4142.TW,5.694442
4,2022-07-06,41.700001,39.900002,41.650002,40.349998,14158450,40.349998,4142.TW,6.044676
5,2022-07-07,42.500000,39.000000,41.799999,41.500000,15687665,41.500000,4142.TW,2.850066
6,2022-07-08,42.500000,40.400002,41.700001,40.700001,7614622,40.700001,4142.TW,-1.927709
7,2022-07-11,40.700001,39.650002,40.700001,40.349998,4099870,40.349998,4142.TW,-0.859956


# 分析股價與營收之關係(yoy)

In [136]:
len(a)

24